The task is to forecast the total amount of products sold in every shop for the test set.

******File descriptions**

sales_train.csv - 2013~15년 자료

test.csv - 15년 11월 자료

sample_submission.csv - 그냥 제출 연습용

items.csv - 아이템과 제품에 대한 supplement information

item_categories.csv  - 아이템 카테고리

shops.csv- shop의 supplement info

**Data fields (데이터 안의 column들)**

ID - an Id that represents a (Shop, Item) tuple within the test set

shop_id - unique identifier of a shop

item_id - unique identifier of a product

item_category_id - unique identifier of item category

item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

item_price - current price of an item

date - date in format dd/mm/yyyy

date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 
2015 is 33

item_name - name of item

shop_name - name of shop

item_category_name - name of item category

In [ ]:
!ls ../input/*

기본적으로 무슨 내용이 있는가~ 를 알아보는 코드

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")

다른 곳에서 matplot 쓰던데 나는 쓸즐 모르겠다

ㅋㅋ 이거 받는데 중간에 route가 이상해서 제대로 못찾아서 헤맴ㅋㅋ 실화냐

In [ ]:
item_cat=pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
item=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sub=pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops=pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test=pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

input 다 받음 ^오^

In [ ]:
sales.head(5)

pandas 배운거 복습 해볼겸 해봤는데 잘 돌아가고, 저장이 잘 된 것을 알 수 있다. moving on

In [ ]:
sales

분석 해보자면, date block num -> 각각이 새로운 달을 의미하며,

In [ ]:
print(sales.info())

In [ ]:
item_cat

In [ ]:
item

아이템에서 아이템 카테고리별로 총 개수 정렬을 해보자

In [ ]:
x=item.groupby(['item_category_id']).count()
x=x.sort_values(by='item_id',ascending=False)
x=x.iloc[0:10].reset_index()
x
ax= sns.barplot(x.item_category_id, x.item_id, alpha=0.8)

아이템 카테고리별 개수 계산 가능

Idea => 모든 카테고리별로 없는 빈칸은 0으로 채우기 그래서 계산하면 훨씬 용이할 것 (각 지점마다 있는 것 없는것 있으므로 없는 칸은 다 0으로 initialize)
그 후에
1. 간단한 방법으로 지금까지의 평균을 내는 방법
2. 

In [ ]:
sub

In [ ]:
sum(sales.duplicated())

중복 내용이 있으니 지워줘야 함. 나중에 merge 문제 가능

In [ ]:
sales[sales.duplicated() == True]

In [ ]:
sales.drop_duplicates(inplace = True)

In [ ]:
sales.date

In [ ]:
sns.boxplot(x=sales.item_price)

이거 보니까 item price 하나가 너무 튀어나가서 30만짜리 지워야 됨.. 객관성을 위해서

In [ ]:
sales = sales[sales.item_price <100000]

굿이에요 굿!

In [ ]:
sns.boxplot(x= sales.item_cnt_day)

여기서 item cnt day가 2000일 넘는건 불가능함.. 그리고 음수인 것도 말이 안됨.. 고로 다 지워줘야 함

In [ ]:
sales = sales[sales.item_cnt_day < 800]

In [ ]:
sales = sales[sales.item_cnt_day > 0]

In [ ]:
sales = sales[sales.item_price >= 0]

In [ ]:
item.drop(['item_name'], axis = 1, inplace = True)

In [ ]:
len(test.item_id)

In [ ]:
len(set(test.item_id)-set(sales.item_id))

item_id 가 실제 테스트랑 다른게 363개가 있다. 고로, 0으로 채워야 함!
그리고 총 매출을 보자..

In [ ]:
sales['revenue'] = sales['item_price']* sales['item_cnt_day']

In [ ]:
sales['revenue']

In [ ]:
sales

이제 새로 table을 생성해야 함.

In [ ]:
answer = []

In [ ]:
sales['date_block_num']
sales

In [ ]:
test

In [ ]:
test['date_block_num'] = 34

In [ ]:
test

마지막 달이니까 date block num =34
이제 answer에 